# Unstructured
- https://unstructured.io/
- https://unstructured-io.github.io/unstructured/index.html
- https://docs.unstructured.io/api-reference/api-services/python-sdk


In [17]:
%pip install "unstructured[all-docs]" unstructured-client watermark

  Using cached unstructured-0.13.6-py3-none-any.whl (1.9 MB)
  Using cached unstructured_client-0.22.0-py3-none-any.whl (28 kB)
  Using cached watermark-2.4.3-py2.py3-none-any.whl (7.6 kB)
  Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
  Using cached lxml-5.2.1-cp311-cp311-macosx_10_9_universal2.whl (8.5 MB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached emoji-2.11.1-py2.py3-none-any.whl (433 kB)
  Using cached dataclasses_json-0.6.5-py3-none-any.whl (28 kB)
  Using cached python_iso639-2024.4.27-py3-none-any.whl (274 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.

In [19]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [20]:
from IPython.display import JSON

import json

from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements, elements_to_json

In [21]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [22]:
%watermark --iversions

unstructured       : 0.13.6
unstructured_client: 0.22.0
json               : 2.0.9



In [23]:
import unstructured.partition

help(unstructured.partition)


Help on package unstructured.partition in unstructured:

NAME
    unstructured.partition

PACKAGE CONTENTS
    api
    auto
    common
    csv
    doc
    docx
    email
    epub
    html
    image
    json
    lang
    md
    model_init
    msg
    odt
    org
    pdf
    pdf_image (package)
    ppt
    pptx
    rst
    rtf
    strategies
    text
    text_type
    tsv
    utils (package)
    xlsx
    xml

FILE
    /Users/sudarshan/Documents/yt-code/youtube-stuffs/.venv/lib/python3.11/site-packages/unstructured/partition/__init__.py




In [24]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = "data/gpt4all.pdf"

# Call the partition_pdf function
# Returns a List[Element] present in the pages of the parsed pdf document
elements = partition_pdf(filename)

# Now, elements is a list of all elements present in the pages of the parsed pdf document

In [25]:
elements

In [26]:
len(elements)

134

In [27]:
element_dict = [el.to_dict() for el in elements]
output = json.dumps(element_dict, indent=2)
print(output)

[
  {
    "type": "UncategorizedText",
    "element_id": "b0c5cfcf93a217591e27d5c97845f59b",
    "text": "3 2 0 2",
    "metadata": {
      "coordinates": {
        "points": [
          [
            16.34,
            263.81000000000006
          ],
          [
            16.34,
            303.81000000000006
          ],
          [
            36.34,
            303.81000000000006
          ],
          [
            36.34,
            263.81000000000006
          ]
        ],
        "system": "PixelSpace",
        "layout_width": 595.276,
        "layout_height": 841.89
      },
      "file_directory": "data",
      "filename": "gpt4all.pdf",
      "languages": [
        "eng"
      ],
      "last_modified": "2024-05-03T13:31:00",
      "page_number": 1,
      "filetype": "application/pdf"
    }
  },
  {
    "type": "Title",
    "element_id": "d71e9973e25dde0d96dc422b5a8fd429",
    "text": "v o N 6",
    "metadata": {
      "coordinates": {
        "points": [
          [
      

In [28]:
unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{'NarrativeText', 'Title', 'UncategorizedText'}


##### We don't see `Table`, table information is not extracted as we expected, lets use different strategy.

### Table extraction from PDF
- Now let’s say that your PDF has tables and let’s say you want to preserve the structure of the tables. 
- You will have to specify the [strategy](https://unstructured-io.github.io/unstructured/best_practices/strategies.html) parameter as `hi_res`. This will use a combination of computer vision and Optical Character Recognition (OCR) to extract the tables and maintain the structure. 
It will return both the text and the html of the table. This is super useful for rendering the tables or passing to a LLM.

> Note: For even better table extraction Unstructured offers an API that improves upon the existing open source models.

> Depending upon machine, you might face different module / library issues, these links might help
- https://stackoverflow.com/questions/59690698/modulenotfounderror-no-module-named-lzma-when-building-python-using-pyenv-on
- https://unstructured-io.github.io/unstructured/installation/full_installation.html

In [ ]:
# 1st way

from unstructured.partition.auto import partition

elements = partition(filename=filename,
                     strategy='hi_res',
           )

tables = [el for el in elements if el.category == "Table"]

print(tables[0].text)
print(tables[0].metadata.text_as_html)

In [ ]:
# 2nd way 

from unstructured.partition.pdf import partition_pdf

elements = partition_pdf(filename=filename,
                         infer_table_structure=True,
                         strategy='hi_res',
           )

tables = [el for el in elements if el.category == "Table"]

print(tables[0].text)
print(tables[0].metadata.text_as_html)

### Now, lets use python sdk to do the same -> [link](https://unstructured-io.github.io/unstructured/api.html)
- For SAAS API, visit this [link](https://unstructured.io/api-key-hosted)
- For Free Unstructured API, visit this [link](https://docs.unstructured.io/api-reference/api-services/free-api)

In [29]:
%pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
from dotenv import load_dotenv, find_dotenv

# Load the .env file
load_dotenv(find_dotenv())

True

In [33]:
saas_api_key_auth = os.environ.get('saas_api_key_auth')
saas_server_url = os.environ.get('saas_server_url')
free_api_key_auth = os.environ.get('free_api_key_auth')

In [34]:
client = UnstructuredClient(
    api_key_auth=free_api_key_auth
    #api_key_auth=saas_api_key_auth,
    #server_url=saas_server_url,
)

In [21]:
# 3rd way 

with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(),
        file_name=filename,
    )

req = shared.PartitionParameters(
    files=files,
    strategy="hi_res",
    hi_res_model_name="yolox",
    skip_infer_table_types=[],
    pdf_infer_table_structure=True,
)

try:
    resp = client.general.partition(req)
    elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)

In [35]:
tables = [el for el in elements if el.category == "Table"]

In [25]:
tables

In [24]:
len(tables)

1

In [23]:
tables[0].text

'Model BoolQ PIQA HellaSwag WinoG. ARC-e ARC-c OBQA Avg. GPT4All-J 6B v1.0* GPT4All-J v1.1-breezy* GPT4All-J v1.2-jazzy* GPT4All-J v1.3-groovy* GPT4All-J Lora 6B* GPT4All LLaMa Lora 7B* GPT4All 13B snoozy* GPT4All Falcon Nous-Hermes (Nous-Research, 2023b) Nous-Hermes2 (Nous-Research, 2023c) Nous-Puffin (Nous-Research, 2023d) Dolly 6B* (Conover et al., 2023a) Dolly 12B* (Conover et al., 2023b) Alpaca 7B* (Taori et al., 2023) Alpaca Lora 7B* (Wang, 2023) GPT-J* 6.7B (Wang and Komatsuzaki, 2021) LLama 7B* (Touvron et al., 2023) LLama 13B* (Touvron et al., 2023) Pythia 6.7B* (Biderman et al., 2023) Pythia 12B* (Biderman et al., 2023) Fastchat T5* (Zheng et al., 2023) Fastchat Vicuña* 7B (Zheng et al., 2023) Fastchat Vicuña 13B* (Zheng et al., 2023) StableVicuña RLHF* (Stability-AI, 2023) StableLM Tuned* (Stability-AI, 2023) StableLM Base* (Stability-AI, 2023) Koala 13B* (Geng et al., 2023) Open Assistant Pythia 12B* Mosaic MPT7B (MosaicML-Team, 2023) Mosaic mpt-instruct (MosaicML-Team, 202

In [30]:
tables[0].metadata

### Now, comes the most interesting part ( utilizing the extracted data in most efficient way)

- It's helpful to have an HTML representation of the table so that you can the information to an LLM while maintaining the table structure.

In [26]:
table_html = tables[0].metadata.text_as_html

In [27]:
table_html

'<table><thead><th>Model</th><th>BoolQ</th><th>PIQA</th><th>HellaSwag</th><th>WinoG.</th><th>ARC-e</th><th>ARC-c</th><th>OBQA</th><th>Avg.</th></thead><tr><td>GPT4All-J 6B v1.0*</td><td>73.4</td><td>74.8</td><td>63.4</td><td>64.7</td><td>549</td><td>36</td><td>40.2</td><td>58.2</td></tr><tr><td>GPT4All-J v1.1-breezy*</td><td>74</td><td>75.1</td><td>63.2</td><td>63.6</td><td>554</td><td>349</td><td>38.4</td><td>57.8</td></tr><tr><td>GPT4All-J v1.2-jazzy*</td><td>74.8</td><td>74.9</td><td>63.6</td><td>63.8</td><td>56.6</td><td>353</td><td>41</td><td>58.6</td></tr><tr><td>GPT4All-J v1.3-groovy*</td><td>73.6</td><td>743</td><td>63.8</td><td>63.5</td><td>577</td><td>35</td><td>38.8</td><td>58.1</td></tr><tr><td>GPT4All-J Lora 6B*</td><td>68.6</td><td>75.8</td><td>66.2</td><td>63.5</td><td>56.4</td><td>357</td><td>40.2</td><td>58.1</td></tr><tr><td>GPT4All LLaMa Lora 7B*</td><td>73.1</td><td>77.6</td><td>72.1</td><td>67.8</td><td>511</td><td>40.4</td><td>40.2</td><td>60.3</td></tr><tr><td>GP

In [28]:
# view what the HTML in the metadata field looks like

from io import StringIO 
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(table_html)
tree = etree.parse(file_obj, parser)
print(etree.tostring(tree, pretty_print=True).decode())

<table>
  <thead>
    <th>Model</th>
    <th>BoolQ</th>
    <th>PIQA</th>
    <th>HellaSwag</th>
    <th>WinoG.</th>
    <th>ARC-e</th>
    <th>ARC-c</th>
    <th>OBQA</th>
    <th>Avg.</th>
  </thead>
  <tr>
    <td>GPT4All-J 6B v1.0*</td>
    <td>73.4</td>
    <td>74.8</td>
    <td>63.4</td>
    <td>64.7</td>
    <td>549</td>
    <td>36</td>
    <td>40.2</td>
    <td>58.2</td>
  </tr>
  <tr>
    <td>GPT4All-J v1.1-breezy*</td>
    <td>74</td>
    <td>75.1</td>
    <td>63.2</td>
    <td>63.6</td>
    <td>554</td>
    <td>349</td>
    <td>38.4</td>
    <td>57.8</td>
  </tr>
  <tr>
    <td>GPT4All-J v1.2-jazzy*</td>
    <td>74.8</td>
    <td>74.9</td>
    <td>63.6</td>
    <td>63.8</td>
    <td>56.6</td>
    <td>353</td>
    <td>41</td>
    <td>58.6</td>
  </tr>
  <tr>
    <td>GPT4All-J v1.3-groovy*</td>
    <td>73.6</td>
    <td>743</td>
    <td>63.8</td>
    <td>63.5</td>
    <td>577</td>
    <td>35</td>
    <td>38.8</td>
    <td>58.1</td>
  </tr>
  <tr>
    <td>GPT4All-J Lora 6B*</td

In [29]:
# let's display this table

from IPython.core.display import HTML
HTML(table_html)

GPT4All-J 6B v1.0*,73.4,74.8,63.4,64.7,549,36,40.2,58.2
GPT4All-J v1.1-breezy*,74,75.1,63.2,63.6,554,349,38.4,57.8
GPT4All-J v1.2-jazzy*,74.8,74.9,63.6,63.8,56.6,353,41,58.6
GPT4All-J v1.3-groovy*,73.6,743,63.8,63.5,577,35,38.8,58.1
GPT4All-J Lora 6B*,68.6,75.8,66.2,63.5,56.4,357,40.2,58.1
GPT4All LLaMa Lora 7B*,73.1,77.6,72.1,67.8,511,40.4,40.2,60.3
GPT4All 13B snoozy*,833,79.2,75,71.3,60.9,442,434,65.3
GPT4All Falcon,77.6,79.8,74.9,70.1,67.9,43.4,42.6,65.2
"Nous-Hermes (Nous-Research, 2023b)",79.5,78.9,80,71.9,74.2,50.9,46.4,68.8
"Nous-Hermes2 (Nous-Research, 2023c)",83.9,80.7,80.1,71.3,75.7,52.1,46.2,70.0
"Nous-Puffin (Nous-Research, 2023d)",815,80.7,80.4,72.5,77.6,50.7,45.6,69.9


#### Now, lets plugin in LangChain to summarize these tables using `Llama3` via `Ollama`
#### [Ollama Playlist](https://www.youtube.com/playlist?list=PLz-qytj7eIWX-bpcRtvkixvo9fuejVr8y)

In [35]:
%pip install langchain langchain_core langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 10.3 MB/s eta 0:00:00m eta 0:00:010:01
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl (21 kB)

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
from langchain_community.chat_models import ChatOllama
from langchain_core.documents import Document
from langchain.chains.summarize import load_summarize_chain

In [37]:
ChatOllama??

Init signature:
ChatOllama(
    *,
    name: Optional[str] = None,
    cache: ForwardRef('Union[BaseCache, bool, None]') = None,
    verbose: bool = None,
    callbacks: ForwardRef('Callbacks') = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    custom_get_token_ids: Optional[Callable[[str], List[int]]] = None,
    base_url: str = 'http://localhost:11434',
    model: str = 'llama2',
    mirostat: Optional[int] = None,
    mirostat_eta: Optional[float] = None,
    mirostat_tau: Optional[float] = None,
    num_ctx: Optional[int] = None,
    num_gpu: Optional[int] = None,
    num_thread: Optional[int] = None,
    num_predict: Optional[int] = None,
    repeat_last_n: Optional[int] = None,
    repeat_penalty: Optional[float] = None,
    temperature: Optional[float] = None,
    stop: Optional[List[str]] = None,
    tfs_z: Optional[float] = None,
    top_k: Optional[int] = None,
    top_p: Optional[float] = None,
    system: Optional[str] = None,
 

In [43]:
llm = ChatOllama(model="llama3")
chain = load_summarize_chain(llm, chain_type="stuff")
output = chain.invoke([Document(page_content=table_html)])

In [44]:
output

{'input_documents': [Document(page_content='<table><thead><th>Model</th><th>BoolQ</th><th>PIQA</th><th>HellaSwag</th><th>WinoG.</th><th>ARC-e</th><th>ARC-c</th><th>OBQA</th><th>Avg.</th></thead><tr><td>GPT4All-J 6B v1.0*</td><td>73.4</td><td>74.8</td><td>63.4</td><td>64.7</td><td>549</td><td>36</td><td>40.2</td><td>58.2</td></tr><tr><td>GPT4All-J v1.1-breezy*</td><td>74</td><td>75.1</td><td>63.2</td><td>63.6</td><td>554</td><td>349</td><td>38.4</td><td>57.8</td></tr><tr><td>GPT4All-J v1.2-jazzy*</td><td>74.8</td><td>74.9</td><td>63.6</td><td>63.8</td><td>56.6</td><td>353</td><td>41</td><td>58.6</td></tr><tr><td>GPT4All-J v1.3-groovy*</td><td>73.6</td><td>743</td><td>63.8</td><td>63.5</td><td>577</td><td>35</td><td>38.8</td><td>58.1</td></tr><tr><td>GPT4All-J Lora 6B*</td><td>68.6</td><td>75.8</td><td>66.2</td><td>63.5</td><td>56.4</td><td>357</td><td>40.2</td><td>58.1</td></tr><tr><td>GPT4All LLaMa Lora 7B*</td><td>73.1</td><td>77.6</td><td>72.1</td><td>67.8</td><td>511</td><td>40.4</t

In [45]:
print(output['output_text'])

It seems like you're sharing a table comparing the performance of various AI models on the SuperGLUE benchmark, with a focus on the StableLM model and its variants.

If I were to summarize this information in a concise manner:

* The StableLM model, when tuned for stability (Stability-Al, 2023), achieves an overall score of 78.6, outperforming some other models.
* The Koala 13B model achieves similar scores to the StableLM model, but with slightly better results on certain tasks.
* Mosaic models generally perform well, especially the MPT7B and mpt-chat variants.
* Wizard models also show promising results, although their performance may be affected by censorship.
* GPT-4 and Falcon 7b models demonstrate strong performance as well.

Let me know if you'd like me to help with any further analysis or insights!


#### Convert to pandas df

In [46]:
%pip install pandas


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
import pandas as pd

# Convert HTML table to pandas DataFrame
dfs = pd.read_html(table_html)

In [54]:
dfs

[                                                Model      BoolQ       PIQA  \
 0                                  GPT4All-J 6B v1.0*       73.4       74.8   
 1                              GPT4All-J v1.1-breezy*         74       75.1   
 2                               GPT4All-J v1.2-jazzy*       74.8       74.9   
 3                              GPT4All-J v1.3-groovy*       73.6        743   
 4                                  GPT4All-J Lora 6B*       68.6       75.8   
 5                              GPT4All LLaMa Lora 7B*       73.1       77.6   
 6                                 GPT4All 13B snoozy*        833       79.2   
 7                                      GPT4All Falcon       77.6       79.8   
 8                  Nous-Hermes (Nous-Research, 2023b)       79.5       78.9   
 9                 Nous-Hermes2 (Nous-Research, 2023c)       83.9       80.7   
 10                 Nous-Puffin (Nous-Research, 2023d)        815       80.7   
 11                  Dolly 6B* (Conover 

In [55]:

# Assuming there's only one table, get the DataFrame
df = dfs[0]

# Now you have the DataFrame
print(df)


                                                Model      BoolQ       PIQA  \
0                                  GPT4All-J 6B v1.0*       73.4       74.8   
1                              GPT4All-J v1.1-breezy*         74       75.1   
2                               GPT4All-J v1.2-jazzy*       74.8       74.9   
3                              GPT4All-J v1.3-groovy*       73.6        743   
4                                  GPT4All-J Lora 6B*       68.6       75.8   
5                              GPT4All LLaMa Lora 7B*       73.1       77.6   
6                                 GPT4All 13B snoozy*        833       79.2   
7                                      GPT4All Falcon       77.6       79.8   
8                  Nous-Hermes (Nous-Research, 2023b)       79.5       78.9   
9                 Nous-Hermes2 (Nous-Research, 2023c)       83.9       80.7   
10                 Nous-Puffin (Nous-Research, 2023d)        815       80.7   
11                  Dolly 6B* (Conover et al., 2023a

In [56]:
df.shape

(38, 9)

In [57]:
df.head()

,Model,BoolQ,PIQA,HellaSwag,WinoG.,ARC-e,ARC-c,OBQA,Avg.
0,GPT4All-J 6B v1.0*,73.4,74.8,63.4,64.7,549,36,40.2,58.2
1,GPT4All-J v1.1-breezy*,74,75.1,63.2,63.6,554,349,38.4,57.8
2,GPT4All-J v1.2-jazzy*,74.8,74.9,63.6,63.8,56.6,353,41,58.6
3,GPT4All-J v1.3-groovy*,73.6,743,63.8,63.5,577,35,38.8,58.1
4,GPT4All-J Lora 6B*,68.6,75.8,66.2,63.5,56.4,357,40.2,58.1
